In [1]:
# this is just some fun including a whole bunch of stuff on this to experiment with this
# it's not directly related to the phd but hopefully will be fun
# and to try rolling and running with julia
# so ultimately the aim is to do prisoners dilemna simulations with agents nad the like in an environment including 
# to see how the population of agents works which is great, and need reprduction and so forth to change the population...
# which should be fun!

# this isn't using the full generics power of julia, but it doesn't need to be generic this, so it doesn't actually matter
# the 

mutable struct Game
    id::Int
    otherAgentId::Int
    otherAgentGenes::String
    myMoves::Array{Int}
    otherMoves::Array{Int}
end
    

mutable struct Agent
    strategy::Function
    genes::String
    id::Int
    Type::Int
    memory::Array{Game}
    currentGame::Game
    LastReward::Int
    totalReward::Int
    gamesPlayed::Int
end

const GENETIC_CODE_LENGTH = 10
const REWARD_TO_REPRODUCE = 50

function count_type_prevalences(as::Array{Agent}, num_types::Int)
    vals = zeros(Int, num_types)
    for a in as
        vals[a.Type] +=1
    end
    return vals
end

function generate_genetic_code()
    # of some arbitrary length
    code = ""
    for i in 1:GENETIC_CODE_LENGTH
        r = rand()
        if r < 0.5
            code *= "0"
        else
            code *="1"
        end
    end
    return code
end


    
function create_agent_population(type_fractions::Array{Int,1}, strategy_dict, num_agents)
    pop = []
    totalId::Int = 0
    for i,frac in enumerate(type_fractions)
        num::Int = num_agents // frac
        as = [Agent(strategy_dict[i],generate_genetic_code(), j+totalId,i,[], nothing,0,0) for j in 1:num]
        hcat(pop, as)
        totalId += num
    end
    return pop
end

# this might be able to be simplified as it's a simple sum, but who knows
function relatedness(a1::Agent, a2::Agent)
    score = 0
    for (s1,s2) in zip(a1.genes, a2.genes)
        if s1 == s2
            score +=1
        end
    end
    return score
end

# this can probably be refactored?
function mutate_code(s, frac)
    for i,el in enumerate(s)
        r = rand()
        if r >= frac
            if el == "0"
                s[i] = "1"
            elseif el=="1"
                s[i]= "0"
            end
        end
    end
    return s
end

relatedness(a1::Agent, a2::Agent) = sum([s1==s2 for (s1,s2) in zip(a1.genes, a2.genes)])
# this is it in the functional paradigm, which is a whole lot easier to understand!
reproduce(a1::Agent, reproduction_frac::Agent) = Agent(strategy_dict[a1.Type], mutate_code(a1.genes, reproduction_frac),0,a1.Type,[],nothing,0,0)
reproduce(a1::Agent) = reproduce(a1, 0.5)
isClone(a1::Agent, a2::Agent) = a1.genes == a2.genes
numClones(a::Agent, pop::Array{Agent}) = sum([a.genes == agent.genes for agent in pop])
fracClones(a::Agent, pop::Array{Agent}) = numClones(a,pop) / length(pop)

# I'm not sure how to delete items from the array, which is annoying, dagnabbit, i should look at this?


    



function PrisonersDilemnaGame(a1::Agent, a2::Agent, num_epochs::Int, num_games::Int)
    a1game = Game(num_games+1, a2.id, [],[])
    a2game = Game(num_games+1, a1.id, [],[])
    a1.current_game = a1game
    a2.current_game = a2.game
    for i in 1:num_epochs
        a1move = a1.strategy(a1)
        a2move = a2.strategy(a2)
        # prisoners dilemna rules
        if a1move == 0 and a2move == 0
            a1.LastReward = 1
            a2.LastReward = 1
        end
        if a1move == 0 and a2move == 1
            a1.LastReward = -10
            a2.LastReward = 1
        end
        if a1move == 1 and a2move == 0
            a1.LastReward = 1
            a2.LastReward = -10
        end
        if a1move == 1 and a2move == 1
            a1.LastReward = -10
            a2.LastReward = -10
        end
        # push the moves
        push!(a1move, a1.currentGame.myMoves)
        push!(a2move, a1.currentGame.otherMoves)
        push!(a1move, a2.currentGame.otherMoves)
        push!(a2move, a1.currentGame.myMoves)
        a1.gamesPlayed +=1
        a2.gamesPlayed +=1
    end
end

#strategies
defectBot(a::Agent) = 1
cooperateBot(a::Agent) = 0

function titForTat(a::Agent)
    if length(a.currentGame.otherMoves) == 0
        return 0 # cooperate on first move!
    end
    prevMove = last(a.currentGame.otherMoves)
    if prevMove == 1
        return 0
    else
        return 1
    end
end

function pavlov(a::Agent)
    # if rewarded last time keep doing what you are doing, else switch!
    if length(a.currentGame.otherMoves) == 0
        return 0 # cooperate on first move!
        end 
    lastMove = last(a.currentGame.myMoves)
    if a.LastReward >=0
        return lastMove
    else
        if lastMove == 0
            return 1
        else
            return 0
        end
    end
end

function cooperate_with_relatives(a::Agent, relatednessThreshold)
    # relatedness threshold hasto be set somehow?
    # not sure how... this is presumably why classes are needed? so I can pass in these parameters arbitrarily!
    rel = relatedness(a.currentGame.otherAgentGenes)
    if rel > relatednessThreshold
        return 0
    else
        return 1
    end
end

function forgivingTitForTat(a::Agent, forgivenessThreshold)
    l = length(a.currentGame.myMoves)
    if length(a.currentGame.otherMoves) == 0
        return 0 # cooperate on first move!
    end 
    if sum(a.currentGame.myMoves[l-forgivenessThreshold, l]) >=forgivenessThreshold
        # i.e. after N continuous defections
        return 0
    else
        return titForTat(a)
    end
end

function randomChoice(a::Agent, prob_defect = 0.5)
    r = rand()
    if r >= prob_defect
        return 0
    else
        return 1
    end
end

function forgiving_titForTat_cooperate_relatives(a::Agent, relatednessThreshold, forgivenessThreshold)
    rel = relatedness(a.currentGame.otherAgentGenes)
    if rel > relatednessThreshold
        return 0
    else
        return forgivingTitForTat(a, forgivenessThreshold)
    end
end

function titForTat_cooperate_relatives(a::Agent, relatednessThreshold, forgivenessThreshold)
    rel = relatedness(a.currentGame.otherAgentGenes)
    if rel > relatednessThreshold
        return 0
    else
        return titForTat(a)
    end
end

function pavlov_cooperate_relatives(a::Agent, relatednessThreshold)
    rel = relatedness(a.currentGame.otherAgentGenes)
    if rel > relatednessThreshold
        return 0
    else
        return pavlov(a)
    end
end

# I need a way to look at clusters in the gene pool... which seems important... one obvious thing to do is cloning
# and that will make it really simple

# here agents don't die, but they can reproduce
function run_experiment(num_games, type_fracs, num_agents, strategy_dict)
    #generate population
    pop = create_agent_population(type_fracs, strategy_dict, num_agents)
    
    
    # how is this going to work... plenty of procrastination, dagnabbit!
    
    

# now we get a population of agents and update it according to the types

            
            
        

LoadError: syntax: extra token "a2move" after end of expression

In [2]:
a = [1 2 4]
b = [3 55 7]
hcat(a,b)

1×6 Array{Int64,2}:
 1  2  4  3  55  7

In [3]:
@code_warntype
a = 2
a += 4.5

LoadError: MethodError: no method matching @code_warntype(::LineNumberNode, ::Module)
Closest candidates are:
  @code_warntype(::LineNumberNode, ::Module, !Matched::Any) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/InteractiveUtils/src/macros.jl:94